# 연구 목표
---

- 데이터 
  - MNIST 손글씨 이미지 데이터 사용
- 목적
  - 손글씨 이미지를 분류(0~9)
- 엔진
  - 텐서플로우
- 모델링
  - 딥러닝 신경망중 CNN 모델 사용
- 평가
  - 예측 모델을 생성하고, 정확도를 이용하여 평가를 수행한다 (분류)

# 데이터 확보
---

- 출처 
    - 텐서플로우의 케라스에서 제공되는 MNIST 데이터를 사용



### 케라스 이용한 MNIST 
---

In [0]:
# 텐서플로우 불러오기 
import tensorflow as tf
%tensorflow_version 1.x

In [0]:
# 데이터를 로드
mnist = tf.keras.datasets.mnist.load_data(path='mnist.npz')

In [0]:
# 로드한 데이터의 타입을 확인
type(mnist), len(mnist), len(mnist[0]), len(mnist[1])

In [0]:
# 길이 확인
len(mnist[0][0]),len(mnist[0][1]), len(mnist[1][0]),len(mnist[1][1])

In [0]:
# dir(mnist)

In [0]:
# 6:4
# 42000 : 28000

In [0]:
# 각 데이터 확인 
mnist[0][0].shape , mnist[0][1].shape,mnist[1][0].shape,mnist[1][1].shape

In [0]:
mnist[0][1][:5], mnist[0][0][:5]

In [0]:
# 케라스에서 제공하는 데이터는 쉽게 로드 되지만, 원본 데이터이기 때문에 => 가공 할 부분이 존재 
# 가공 데이터를 받아서 처리(1.x)  
from tensorflow.examples.tutorials.mnist import input_data

In [0]:
mnist = input_data.read_data_sets('./data/mnist/', one_hot=True)

In [0]:
type(mnist)

In [0]:
# 훈련 데이터
mnist.train.images.shape, mnist.train.labels.shape

In [0]:
# 테스트 데이터
mnist.test.images.shape, mnist.test.labels.shape

In [0]:
mnist.train.images

# 데이터 준비

In [0]:
#  이미지 한개당 특징(feature)의 크기 
pixels = mnist.train.images.shape[1]
pixels

In [0]:
# 레이블 특징(feature)의 크기
nums = mnist.train.labels.shape[1]
nums

In [0]:
import numpy as np
# 이미지 1개당 가로 혹은 세로 크기 (정사각형이므로 동일)
pixel_wh = int( np.sqrt( pixels ) )   # 수치는 정수로 맟춰야 한다. 
pixel_wh

In [0]:
# 정답 레이블에서 원래 값 0~9까지 찾아내는 방법 
# array([[0., 0., 0., 0., 0., 0., 0., 1., 0., 0.]])  =>> '7'
# 나중에 정답을 출력 
mnist.train.labels[:1]

In [0]:
print(mnist.train.labels[2])

In [0]:
mnist.train.labels[2].argmax()

In [0]:
mnist.train.labels[0]

In [0]:
np.where(mnist.train.labels[0])[0]

In [0]:
np.where(mnist.train.labels[0])[0][0]

# 모델 구축, 딥러닝
---

## 1. 모델 설계, 레이어 설계 -> 모델링
---

``` bash
layers
  L 입력층
  L 합성곱층
  L 풀링층 
  L 합성곱층
  L 풀링층 
  L 전결합층
  L 드롭아웃층
  L 출력층
  
```

## 2. 데이터 플로우 그래프 구축
---


### 입력층 

- x 
- 손글씨 이미지가 데이터로 주입된다.
- 플레이스홀더 구성 
- shape : (None, 784=pixels)

In [0]:
x = tf.placeholder(tf.float32 , shape =(None, pixels), name = 'x' )
x

### 합성곱층 1
---

- 외부에서, 이전단계 레이어에서 데이터가 흘러들어온다
- 데이터 주입? ( X )
- 구성원
  - W:공용파라미터, 가중치 => 필터, 커널
  - b:편향
  - s:stride
  - p:padding

In [0]:
# 가중치 필터 W를 만드는 함수 
# 가중치 필터는 언제나 3x3의 크기를 가진다가 아니다! 
# shape : W의 shape
# name  : W의 이름(텐서보드에서 해당 텐서들을 확인/ 구분용으로 사용)

def makeWeightVariable(shape, name):
    # 변수 초기값 -> 절단 평균 분포를 통해서 난수값으로 초기화 
    # 0.1은 설정값 -> 가중치 필터의 값은 일단 난수로 설정하였다.
    init_d = tf.truncated_normal(shape, stddev= 0.1) # 초기값

    # W를 생성
    W = tf.Variable(init_d, name = 'W_'+name) 
    
    # 가중치 필터(커널) W를 리턴
    return W 

In [0]:
# 편향을 만드는 함수 
def makeBiasVariable(shape, name):
    # 상수로 값을 하나 고정하여서 적용 
    init_b = tf.constant( 0.1, shape=[shape] ) # 이부분 참조
    b      = tf.Variable( init_b, name='b_' + name )

    # 편향 값을 리턴 받는다.
    return b

    # 참조 - https://www.tensorflow.org/api_docs/python/tf/constant

In [0]:
# 합성곱층을 만드는 함수 
def makeConv2d(x, W, name):
    # 한칸식 stride
    conv2d = tf.nn.conv2d(x, W, strides=[1,1,1,1], padding= "SAME", name = 'conv_'+name )
    
    return conv2d


In [0]:
'''
tf.nn.conv2d(
  input  : [
            batch,         # 1회 훈련시 들어오는 데이터의 총개수:이미지 총개수
            in_height,     # 이미지의 세로크기 
            in_width,      # 이미지의 가로크기
            in_channels    # 입력이미지의 채널수, 1 or 3 
            ]
  filters: [
            filter_height,    # 필터의 세로 : ex) 3
            filter_width,     # 필터의 가로 : ex) 3
            in_channels,      # 입력 채널수 : 1
            out_channels      # 출력 채널수 : 알아서 설정(마음대로, 설정값)
            ]
  strides: An int or list of ints that has length 1, 2 or 4
            # 정수값, 정수 값들의 리스트로 구성 
            # 구성값은 1 or 2 or 4를 사용 
            [ bacth : bacth와 depth는 1을 사용한다 : 1 
              W     : w와 h는 통상 값은을 부여 
              h     : 1
              depth : 1
            ]
  padding: Either the string "SAME" or "VALID"
            # 특정 맵을 보정한다. 
            # 자르다 보면 외곽선 부분이 부적하여서 필터 작업이 안될수 있으므로 이를 보정하는 설정 
            # 통상적으로 0으로 세팅 
            # "SAME":같은크기로 보정한다 혹은 "VALID":유효한 부분만 보정한다
'''

### 합성곱층 연결
---

- x, W, b등을 연결하여 활성화 맵으로 출력할 수 있도록 데이터 플로우 그래프를 정의한다
- 텐서보드상에서 이 합성곱을 하나의 노드로 표현하기 위해, name_scope 하나의 이름이 미치는 공간, 관계, 형태를 규정

In [0]:
# 합성곱층 1 생성, 입력대비 출력까지의 모든 관계(그래프)를 표현 

with tf.name_scope('conv1') as scope:
    # W
    # [필터의 높이, 필터의 가로, 입력 채널 수, 출력 채널수 ]  => [5, 5, 1, 32 ]
    # 출력채널수를 32라고 지정하면 이미지 한장을 넣어서, 결과물이 32가 나온다. 
    # 1(입력 채널 수)을 제외하고 모두 가정값이다 => 5, 5, 32는 모두 설정값이다(실험치이다)
    W = makeWeightVariable( [5, 5, 1, 32 ] , 'conv1' )

    # b
    b = makeBiasVariable(32 , 'conv1')

    # x : 입력층 (None, 784) => (batch(배치), h(세로), w(가로), channel(채널))
    # 입력층의 모양을 tf.nn.conv2d에서 원하는 모양으로 맞춰준다
    x_imgs  = tf.reshape(x, (-1, pixel_wh, pixel_wh, 1))

    # conv1
    h_conv1 = tf.nn.relu( makeConv2d( x_imgs, W,'conv1') + b )
    # h_conv1 = tf.nn.relu( 피처맵 )
    # h_conv1 : activation Map


    # 텐서 실행시에는 한번에 출력해야한다 -> 대화형이기때문에 기록이 남는다.

In [0]:
h_conv1

# 합성곱층을 통과하면 1개의 이미지가 32개가 된다 
# 이유는 가중치 필터에서 출력 채널수를 32로 했기때문 

- 이미지 수량 

    - 합성공층을 통과하면 1개의 이미지가 32개가 된다.
    - 이유는 가중치 필터에서 출력 채널수를 32개로했기 때문에 수량이 증가했다.  

- 이미지 크기 
    - 현재 결과물은 (?, 28, 28, 32) 이다.
    - 이미지의 세로, 가로의 크기가 원본과 동일하다 
        - padding = 'SAME' 때문에 크기가 되었다
    - VAILD로 구현하면 (?, 24, 24, 32) 크기가 줄어든다

In [0]:
h_conv1.shape

### 풀링층 1
---

- 특성(특성맵, 활성화맵)을 강화한다.
- 최대/평균/최소 풀링 제공, 구현이 가능하다.
- 샘플링 표현으로도 묘사
- stride, padding이 없다
    - 풀링의 수행은 shape이 작아지는 결과를 얻게된다.
    - 내부적으로 풀링 연산을 수행하기 위해서는 얼만큼 이동하여서 값을 추출해야 하는지 보폭의 양이 존재해야한다. 
    - 커널이 없더라도, 연산에 필요한 스트라이드 값은 필요하다.
    - tride가 있다면, padding도 존재해야한다

- 입력 : h_conv1 (?, 28,28,32)

In [0]:
# 풀링함수  -> 최대풀링 
def makeMaxPooling( x ):
    # 결과물의 포멧은 NHWC이다(현재 상황의 기본값)
    return tf.nn.max_pool( x, ksize = [1, 2, 2, 1], strides = [1 ,2, 2, 1], padding = 'SAME'  )
'''
input   : 4-D tensor, 이전 단계의 산출물, [ batch, h, w, channels ]
ksize   : [batch, h, w, 출력채널수 ] => 기본 커널에 대비해서 W는 없다
          이중에 최대값, 이중에 평균값, 이중에 최소값등의 선택
          => 데이터 1개 대상 => 각 차원의 크기 지정했다 
          => 최대풀링값이 1개 나온다 => 4번째 1의 의미
          => 입력이 1개이다=> 첫번째 1의 의미 
          => 가운데 2, 2만 신경쓰면 된다
strides : [batch, w, h, depth], batch=depth, w or h : 1, 2, 4등 사용
          => 가운데 w, h가 최종 산출물의 크기에 영향을 미친다
padding : 'SAME', 'VAILD'
'''


In [0]:
# 풀링층 1 생성 
with tf.name_scope('pool1') as scope:
    h_pool1 = makeMaxPooling( h_conv1 )

In [0]:
h_pool1.shape, h_pool1
# (?, 14, 14, 32) <- 크기가 반으로 줄어들었다, 출력채널수는 입력대비 동일하다

### 합성곱층 2
---

- 입력 : h_pool1
- 출력 : h_conv2 => (?, 14, 14, 64)


In [0]:
name_conv2 = 'conv2'
with tf.name_scope(name_conv2) as scope:
    # [5(h), 5(w), 32(입력채널수=이전단계의출력채널수), 64(최종 출력채널수)]
    W_conv2 = makeWeightVariable( [5, 5, 32, 64 ], name_conv2 )
    b_conv2 = makeBiasVariable( 64, name_conv2 )
    h_conv2 = tf.nn.relu( makeConv2d( h_pool1, W_conv2, name_conv2 ) + b_conv2 )

In [0]:
h_conv2


### 풀링층 2 
---

- 최대 풀링을 사용하여 아래와 같은 입력 shape을 아래에 묘사된 출력 shape과 동일하게 처리하시오
    - 입력 : h_conv2 => ( ?, 14, 14, 64)
    - 출력 : h_pool2 => ( ?, 7, 7, 64 )

In [0]:
# shape을 다양하게 하고 싶다면 => ksize,나 strides등등을
# 인자로 입력받고 처리하겠금 함수를 수정하면 된다. 

with tf.name_scope('pool2') as scope:
    h_pool2 = makeMaxPooling( h_conv2 )

### 전결합층
---

- 출력층으로가는 직전 단계이다 
- 전결합층에 도착한 입력데이터 h_pool2는 Tensor => 행렬이다 => 출력층에 보내기 위해서 Flatten을 수행한다 => h, w, channel로 구성된 데이터를 한개릐 row로 펴준다
- 현재 이미지 1개는 => 7 \* 7 \* 64 = 3136개로 정보로 표현되고 있다 => 1024로 flatten 한다 (정보를 압축(3136->1024)해서 펴준다)
- 이미지 사이즈는 => 풀링을 2회 거치면서 조정 되었다 => (28/2)/2 => 7
- 1024는 설정값

- 입력 : h_pool2 => (?, 7, 7, 64)
- 입력의 D -> 4d=>2d로 reshape
- 출력 : h_fc => (?, 1024)

In [0]:
with tf.name_scope('fully_conected') as scope:
    
    num = 7 * 7 * 64 # h_pool2 정보에서 뽑아서 계산하시오(변경)
    # 가중치 필터는 합성 곱을 위한것이 아니라, 행렬의 곱을 위해서 모양을 맞췄고
    # 입력값에 가중치를 곱하다는 퍼셉트론에 맞춰서 계산 
    W_flat = makeWeightVariable([num, 1024],'fully_conected')
    h_pool2_flat = tf.reshape(h_pool2,[-1, num])
    h_fc = tf.nn.relu(tf.matmul(h_pool2_flat, W_flat ))


In [0]:
h_fc

- (-1, 3136) 행렬의곱 (3136, 1024) 
  => (-1, 1024)

### 드롭아웃층
---

- 과적합, 과잉적합, over fit <= 방지
- 특정 데이터에 길들어지는, 편향적인 형태를 가진 것을 방지
- 뉴런(특정 신경망의 일부분)을 죽여서, 학습을 못하게 처리(랜덤)


In [0]:
with tf.name_scope('dropout') as scope:
    # keep_prob
    # 값을 외부에서 받아서(데이터를 주입) 신경망을 죽이는 비율을 조정 
    keep_prob = tf.placeholder(tf.float32)
    # h_fc_drop 
    h_fc_drop = tf.nn.dropout(h_fc, rate=1-keep_prob)

In [0]:
h_fc_drop

### 출력층 
---
- 이전 단계의 산출물을 최종레이블의 shape로 맞춘다(수렴한다.)
- 최종 레이블 (10개 , 0~9)
    - (?,10)
- 최종 출력에 데이터를 수렴할때 활성화 함수를 사용할수 있다. 
    - 현재 레이블의 클래스의 수는 10개이므로, 다항이 되고 => softmax() 사용하는게 적절하다
    - softmax()을 통과한 전체 데이터를 합산하면 1이다. 값이 0~1사이로 존재

In [0]:
with tf.name_scope('output') as scope:

    # 1. 입력 -> h_fc_drop
    W_output = makeWeightVariable( [1024,10], 'output' )   # W
    b_output = makeBiasVariable( 10, 'output' )         # b

    # 2. 출력 -> y_conv-> softmax
    # y_conv => tf.nn.softmax( tf.matmul(x,w)+b)
    # x => h_fc_drop
    y_conv = tf.nn.softmax( tf.matmul(h_fc_drop,W_output)+b_output)
    # 결과 -> (?,10)

In [0]:
y_conv

In [0]:
# 정답! -> 데이터를 주입받는다.
y_ = tf.placeholder(tf.float32, shape=(None, nums), name = 'y_')

### 학습 플로우 작성

#### 크로스 엔트로피, 손실값

- 비용을 줄인다, 손실값을 줄인다
- 이런 지표는 원하는 결과에서 얼마나 떨어져 있는지를 보여준다


In [0]:
with tf.name_scope('loss') as scope:
    cross_entropy = - tf.reduce_sum(y_ * tf.log(y_conv))

#### 경사 하강법, 최적화

- 손실값을 줄이는 방향으로, 점진적으로 값을 조정하면서 , 최적화를 수행한다
- 종류 : 종류별로 정리를 한다
    - SGD
    - Adam

In [0]:
with tf.name_scope('agd') as scope:
    optimizer = tf.train.AdamOptimizer()
    train = optimizer.minimize( cross_entropy )


In [0]:
train

### 예측, 평가관련 플로우 작성(정확도)

In [0]:
with tf.name_scope('predict') as scope:
    # 예측 
    predict = tf.equal(tf.arg_max(y_conv, 1 ),tf.arg_max(y_, 1 ))
    # 정확도 
    accuracy = tf.reduce_mean(tf.cast(predict, tf.float32))

## 3. 실행(학습, 예측, 평가)
---


In [0]:
# 주입할 데이터의 모양을 세팅해주는 함수를 구성 

def makeFeedDictParam(imgDatas, labels, prob):
    # 리턴값 : { x: , keep_prob: , y_: } 
    #       => { x: imgDatas , y_: labels , keep_prob:keep_prob } 
    return { x: imgDatas , y_: labels , keep_prob:prob } 

In [0]:
# 1. 세션 열기  
#  학습 횟수 
TRAIN_COUNTS = 3000      # 설정값
ONE_TRAIN_COUNTS = 50    # 한번 훈련시 사용하는 데이터의 양 
VERBOSE_TERM = 100       # 100번째 훈련이 되면 로그를 출력 
with tf.Session() as sess: 
    # 2. 초기화 -> 텐서 변수
    init = tf.global_variables_initializer()
    sess.run(init)  # 초기화 
    # 3. 데이터 준비 
        # 3-1. 훈련용 데이터 주입하기 위한 준비 -> 생략
            # 3-1-1. 종류 :  x, keep_prob, y_(정답) -> { x: , keep_prob: , y_: }
        
        # 3-2. 테스트용 데이터를 주입하기 위한 준비
            # 3-2-1. 종류 :  x, keep_prob, y_(정답) -> 훈련용과 동일 -> { x: , keep_prob: , y_: }
            # 검증 
            # 1 -> rate = 1- keepProb( =1) =0 이되므로 신경을 죽이는 행위는 제거한다.
            # 검증용은 과적합이라는 주제와 연관성이 없으므로 1로 처리
    test_img = mnist.test.images
    test_lab = mnist.test.labels
    keep_prob_size  = 1
    test_feedDict = makeFeedDictParam( test_img, test_lab, keep_prob_size)

    # 4. 반복학습 => 데이터를 나눠서 학습시키겠다 => 학습 횟수를 증가 
    for step in range(TRAIN_COUNTS):   # 0~2999 : 3천번 수행 
        # 4-1. 1회 훈련에 사용하는 데이터 획득(총량, 횟수 고려하여 구성)
        batch = mnist.train.next_batch(ONE_TRAIN_COUNTS)  #  0:이미지 데이터, 1:레이블정답 데이터
        # 4-2. 훈련용 데이터를 생성 
        # 데이터 세트가 50세트 , 과적합 방지용으로 신경을 죽이는 비율 50% 
        train_fdp = makeFeedDictParam( batch[0],  batch[1], 0.5)
        # 4-3. 데이터를 주입하여 훈련한다 
        # 더미값(버리는값), 손실값 = sess.run([train, cross_entropy],feed_dict= train_fdp ) 두개로 나눠서 출력 
        # 작업 2개를 []로 묶어서 구동시켰다. 데이터 주입은 train 이라는
        # 단순한 값을 확인하기위해서 sess.run()의 작업 항목으로 묶어두었다.   
        # 작업이 두개라 결과 역시 두개가 나온다 -> 그중 학습의 결과는 필요없는 더미값이므로
        # _로 받았고 , 손실(LOSS)만 변수로 받았다.
        _, loss =sess.run([train, cross_entropy],feed_dict= train_fdp )
        # 4-4. 특정 횟수마다 로그를 출력한다. -> test_feeddict 사용 
        if step % VERBOSE_TERM == 0 :
            # 4-5. 테스트 데이터를 현재까지 훈련된 신경망에 주입하여, 정확도를 측정 
            acc = sess.run( accuracy, feed_dict = test_feedDict )
            # 4-6. 단계, 정확도, 손실값 
            print('s=%4s, a=%20s, l=%20s' % (step, acc, loss) ) 

    # 5. 최종 결과를 출력  -> test_feeddict 사용 
    acc = sess.run( accuracy, feed_dict = test_feedDict )
    print('-'*50)
    print('최종 결과')
    print( 's=%4s, a=%20s, l=%20s' % (step, acc, loss) )
    print('-'*50)

# 시스템 통합(생략)